<div>
<img src="figures/svtLogo.png"/>
</div>

<center><h1>Mathematical Optimization for Engineers</h1></center>
<center><h2>Bonus exercise 1</h2></center>

In this exercise, you will implement and solve the optimization problem that we formulated in Lab 01.<br>
<br>
Please complete the following tasks as you program:

1. Use the <i>linprog</i> solver from the <i>scipy</i> package.
<br>
<br>
2. Is the demand penalty active in the optimal solution?
<br>
<br>
3. Write down the units of all optimization variables next to their names as comments e.g.


**Hint: The optimal objective value is 1943.87 €/h.**

In [ ]:
# P1 - kW
# PP - kW

You may use the template below, for your code.

In [1]:
import numpy as np
from scipy.optimize import linprog

Tip: every solver expects input in its own unique format. We should inform ourselves of this format by looking at the documentation of the solver <u>before</u> we start to implement our problem.

[Click here for linprog documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html)

`linprog(c, Aub, bub, Aeq, beq, bounds, method='interior-point', options, x0)`


\begin{align}
\min_\mathbf x \ & \mathbf c^T \mathbf x \\
        \mbox{s. t.} \; \ & \mathbf A_{ub} \mathbf x \leq \mathbf b_{ub},\\
        & \mathbf A_{eq} \mathbf x = \mathbf b_{eq},\\
        & \mathbf l \leq \mathbf x \leq \mathbf u ,
\end{align}


where $\mathbf x$ is a vector of decision variables; $\mathbf c$,
$\mathbf b_{ub}$, $\mathbf b_{eq}$, $\mathbf l$, and $\mathbf u$ are vectors; and
$\mathbf A_{ub}$ and $\mathbf A_{eq}$ are matrices.

In [2]:
# dimensions of problem 
numVars = 18
numEqC = 10
numInEqC = 2

# define indicies for variables
I1, I2, HE1, HE2, LE1, LE2, C, BF1, BF2, HPS, MPS, LPS, P1, P2, PP, EP, Power, Fuel = (i for i in range(numVars))

# prices from Table 2
fuel_cost_coeff=1.5*10 ** (-6)#Unit Euro/kJ
condensate_loss_coeff=0.008 # Unit Euro/kg
produced_power_coeff=0.02 # Unit Euro/kWh
purchased_power_coeff=0.05# Unit Euro/kWh
demand_penalty=0.001# Unit Euro/kWh

# enthalpies from Table 4
Enthalpy_of_HPS=3163 #Unit kJ/kg
enthalpy_mps=2949 #Unit kJ/kg
enthalpy_lps=2911 #Unit kJ/kg 
enthalpy_condensate=449 #Unit kJ/kg

# data from Table 5
Evaporator_Efficiency=0.75 #Unit -
basic_power=12000 # Unit kW

In [3]:
# coefficients in the objective function: 
c = np.zeros(numVars)
c[PP]=produced_power_coeff
c[Fuel]=fuel_cost_coeff
c[C]=condensate_loss_coeff
c[EP]=purchased_power_coeff  # case for EP >= 12MW



In [4]:
# inequality constraints
Aub = np.zeros([2,numVars])
bub = np.zeros(2)

# add your code here

# I1-HE1 <= 60000 
Aub[0,I1]=1
Aub[0,HE1]=-1
bub[0]=60000

# P1+P2+EP >= 24500
Aub[1,P1] = -1
Aub[1,P2] = -1
Aub[1,EP] = -1
bub[1] = -24500


In [5]:
# equality constraints 

Aeq = np.zeros([10, numVars])
beq = np.zeros(10)

# add your code here

# Enthalpy_of_HPS*HPS=Evaporator_Efficiency*Fuel
Aeq[0,Fuel]=Evaporator_Efficiency
Aeq[0,HPS]=-Enthalpy_of_HPS

# HPS = I1+I2+BF1
Aeq[1,HPS] = -1
Aeq[1,I1] = 1
Aeq[1,I2] = 1
Aeq[1,BF1] = 1

# I1 = HE1+LE1+C
Aeq[2,I1] = -1
Aeq[2,HE1] = 1
Aeq[2,LE1] = 1
Aeq[2,C] = 1

# I2 = HE2+LE2
Aeq[3,I2] = -1
Aeq[3,HE2] = 1
Aeq[3,LE2] = 1

# HE1+HE2+BF1 = BF2+MPS
Aeq[4,HE1] = 1
Aeq[4,HE2] = 1
Aeq[4,BF1] = 1
Aeq[4,BF2] = -1
Aeq[4,MPS] = -1

# LPS = LE1+LE2+BF2
Aeq[5,LE1] = 1
Aeq[5,LE2] = 1
Aeq[5,BF2] = 1
Aeq[5,LPS] = -1

# 3163*I1 = enthalpy_mps*HE1+2911*LE1+449*C+3600*P1
Aeq[6,I1] = -3163
Aeq[6,HE1] = enthalpy_mps
Aeq[6,LE1] = 2911
Aeq[6,C] = 449
Aeq[6,P1] = 3600

# 3163*I2 = enthalpy_mps*HE2+2911*LE2+3600*P2
Aeq[7,I2] = -3163
Aeq[7,HE2] = enthalpy_mps
Aeq[7,LE2] = 2911
Aeq[7,P2] = 3600

# PP=P1+P2
Aeq[8,PP] = -1
Aeq[8,P1] = 1
Aeq[8,P2] = 1

# Power=PP+EP
Aeq[9,Power] = -1
Aeq[9,PP] = 1
Aeq[9,EP] = 1


In [6]:
# lower bounds and upper bounds for variables
bnds = []
lb = np.zeros(numVars)
ub = np.ones(numVars) * np.inf

lb[P1]=2500
lb[P2]=3000
lb[MPS]=123000
lb[LPS]=45000

ub[P1]=6250
ub[I1]=87000
ub[C]=28000
ub[P2]=9000
ub[I2]=110000
ub[LE2]=64000

# collect bounds in the correct format for using linprog
bnds = []
for i in range(0, len(lb)):
    bnds.append((lb[i], ub[i]))

In [9]:
# case A
# EP >= 12 MW
bnds[EP] = (12000, np.inf)
c[EP]=purchased_power_coeff

linprog(c, Aub, bub, Aeq, beq, bnds, method='interior-point')


/tmp/ipykernel_57/3916137544.py:6: DeprecationWarning: `method='interior-point'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  linprog(c, Aub, bub, Aeq, beq, bnds, method='interior-point')


     con: array([-3.57627869e-06, -1.60071068e-10,  8.94942787e-10, -1.42608769e-09,
        4.65661287e-10,  1.34605216e-10, -6.25848770e-07, -8.04662704e-07,
       -2.18278728e-11, -4.72937245e-11])
     fun: 1951.5020425237421
 message: 'Optimization terminated successfully.'
     nit: 14
   slack: array([4.89033460e+04, 4.75579509e-06])
  status: 0
 success: True
       x: array([7.08523033e+04, 9.98514550e+04, 5.97556493e+04, 6.32443513e+04,
       8.39289573e+03, 3.66071038e+04, 2.70375826e+03, 6.79553640e-05,
       5.47117774e-04, 1.70703758e+05, 1.23000000e+05, 4.50000000e+04,
       6.17797738e+03, 6.32202259e+03, 1.25000000e+04, 1.20000000e+04,
       2.45000000e+04, 7.19914650e+08])

In [13]:
# case B
# EP < 12 MW
bnds[EP] = (0, 12000)
c[EP]=purchased_power_coeff-demand_penalty

result = linprog(c, Aub, bub, Aeq, beq, bnds, method='interior-point')

print(result)
# as we didn't account for the offset in penalty we have to do it manually here
print(result.fun + basic_power*demand_penalty)


     con: array([-2.26497650e-06, -4.58385330e-10,  1.29512046e-09, -4.94765118e-10,
        6.27915142e-09,  2.46101541e-09, -3.18884850e-06,  3.06963921e-06,
       -2.48292054e-10,  3.63797881e-12])
     fun: 1931.8653083236616
 message: 'Optimization terminated successfully.'
     nit: 12
   slack: array([5.62829712e+04, 2.01906296e-05])
  status: 0
 success: True
       x: array([6.17170220e+04, 1.10000000e+05, 5.79999932e+04, 6.50000068e+04,
       6.08240833e-03, 4.49999932e+04, 3.71702274e+03, 2.86628526e-03,
       1.70517367e-03, 1.71717025e+05, 1.23000001e+05, 4.50000009e+04,
       6.24999994e+03, 7.01388882e+03, 1.32638888e+04, 1.12361113e+04,
       2.45000000e+04, 7.24187933e+08])
1943.8653083236616


/tmp/ipykernel_57/2971995474.py:6: DeprecationWarning: `method='interior-point'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  result = linprog(c, Aub, bub, Aeq, beq, bnds, method='interior-point')
